# ch2 類神經網路

在上一章中我們建立一個簡單的網路並運行節點，在這一章中我們將建立一個類神經網路並優化網路。

## 網路架構

我們打算建立一個兩層的類神經網路並使用著名的鳶尾花資料集訓練。

In [1]:
from sklearn import datasets
import numpy as np

iris=datasets.load_iris()     # 載入資料
feature=iris.data             # 輸入的特徵資料
label=np.eye(3)[iris.target]  # 預測的類別資料

下面是我們的網路設計圖。

![ch2-1](https://imgur.com/Y9dRbfz.png)

如上圖所示，我們將建立一個兩層的神經網路。

在資料方面，輸入網路的資料保存到資料節點*data_x*中，並在輸入神經網路前做標準化預處理。

而在神經網路方面，在第一層使用tanh作為activation function，第二層使用softmax作為activation function，然後將結果保存到名稱為pre的節點。最後再將*節點pre*的值與*節點data_y*中保存的標籤資料去計算cross-entropy並將結果保存到*節點loss*中。

我們將依據下面的步驟建立網路:
1. 導入單元
2. 建立節點連結

### 導入單元

在前一章我們使用了一種建立網路後再導入單元的方法，但其實我們可以在網路產生時就先往裡面導入一些單元。

In [2]:
from nyto import net_tool as to
from nyto import layer

# 導入方式一: 在new_net方法中可以直接導入單元
(nn, node)=to.new_net(                  
    # 使用add_data導入資料節點
    data_x=to.add_data(feature),  # 加入資料節點: data_x
    data_y=to.add_data(label),    # 加入資料節點: data_y
    
    # 不使用任何導入方式則預設導入的是模型節點
    layer1=layer.new_nn_layer((4,12)),  # 加入模型節點: layer1(單層神經網路)
    layer2=layer.new_nn_layer((12,3))   # 加入模型節點: layer2(單層神經網路)
)

也可以在網路產生後才往裡面導入單元:

In [3]:
# 導入方式二: 在網路產生後才往裡面導入單元
(nn, node)=to.new_net()

node.data_x=to.add_data(feature)
node.data_y=to.add_data(label)
node.layer1=layer.new_nn_layer((4,12))
node.layer2=layer.new_nn_layer((12,3))

如果導入的資料不需要優化，比如訓練資料或函數之類的。建議使用`net_tool.add_data`導入到資料節點中。可以幫助模型訓練時提升速度和節省記憶體。

### 建立節點連結

導入單元後就可以將剛剛建立的單元節點連結起來組織成網路，這是通常的步驟。但是有些內建的函數可以允許使用者在建立連接時自動導入函數，而不需要事前導入。下面是使用自動導入的例子:

In [4]:
from nyto import unit_function as uf

node.data_x >> uf.col_nor() >> node.output

當使用者使用`uf.col_nor()`在建立連結時，會自動將`_col_nor`函數導入到網路中，然後將新產生的函數的節點用來建立當前連結。

簡而言之，當使用者執行下列腳本時:
    
    node.data_x >> uf.col_nor() >> node.output
    
其實等於是做了這件事:

    node._col_nor = net_tool.add_data(uf._col_nor)
    node.data_x >> node._col_nor >> node.output
    
至於`>>`符號的作用等價於`()`呼叫，你完全可以將上面的腳本改寫成:

    node._col_nor = net_tool.add_data(uf._col_nor)
    node.output = node._col_nor(node.data_x)

其中`data_x`和`_col_nor`是資料節點，而`output`是普通節點。

### 完整代碼

下是建立我們目標網路的完整過程:

In [5]:
from nyto import net_tool as to
from nyto import layer
from nyto import unit_function as uf

# 導入單元
(nn, node) = to.new_net(                  
    data_x = to.add_data(feature),  # 加入資料節點: data_x
    data_y = to.add_data(label),    # 加入資料節點: data_y
    
    layer1 = layer.new_nn_layer((4,12)),  # 加入模型節點: layer1(單層神經網路)
    layer2 = layer.new_nn_layer((12,3))   # 加入模型節點: layer2(單層神經網路)
)

# 建立節點連結
node.layer1_output = node.data_x >> uf.col_nor() >> node.layer1 >> uf.tanh()
node.layer2_output = node.layer1_output >> node.layer2 >> uf.softmax()
node.pre = node.layer2_output
node.loss = uf.cross_entropy(node.pre, node.data_y)

建立成功後我們試試看運行計算loss值的*loss節點*和計算預測值的*pre節點*:

In [6]:
loss, pre = to.get(node.loss, node.pre)

In [7]:
loss # 訓練前的loss值

1.0986122886681096

## 訓練網路

下面我們就來訓練網路，訓練網路可以使得loss值變得更小。
與基於梯度下降訓練的方式不同，nyto中訓練網路基於下面幾個步驟來進行:
1. 建立池
2. 訓練池
3. 查看池

## 建立池

池可以想像為有許多網路的一個集合，我們需要提供三個參數來生成池。分別是:
1. 網路節點(node_if): 需要優化的節點，優化的目的是讓該節點的輸出值最小化。
2. 池大小(pool_size): 越大的池可以得到越穩定的訓練過程，但是需要的運算量也越大。最小為1。
3. 隨機程度(random_size=1): 該值越大，則初始化時池內的網路參數隨機程度越大。該值大於等於0。

下面我們來建立一個用於優化*loss節點*的池:

In [8]:
from nyto import train_tool as train

pool = train.new_pool(node.loss, pool_size=10)

## 訓練池

使用`epso_opt`優化器，可以對池進行1次的優化，使用迴圈多次呼叫則可以多次優化。優化完成後會返回一個新的池。

In [9]:
# 建立優化器
optimizer=train.epso_opt()

# 進行10次優化
for t in range(10):
    pool = optimizer(pool)

## 查看池

通過查看池來取得訓練的結果。使用`net`方法取得訓練好的網路，使用`loss`方法取得節點的輸出值。

查看時，使用`[]`來查看，輸入的數字為對應的排名。比如想查看節點的輸出值最小的網路則呼叫:

In [10]:
pool.net[0]

net(mod={'layer1', 'layer2'}, data={'data_y', 'data_x'})

而想查看該網路的輸出值則呼叫:

In [11]:
pool.loss[0]

0.5419818660475417

同理，想查看第二小的輸出值則呼叫:

In [12]:
pool.loss[1]

0.6318364036630774

## 使用訓練好的網路

當網路訓練好後我們需要將該網路從池中取出並用來預測資料可以這麼做:

In [13]:
best_nn = pool.net[0] # 取得loss最小的網路

接下來我們需要通過網路取得節點，然後我們就能取得該網路的預測值了:

In [14]:
# 取得網路的節點
best_nn_node = to.create_connecter(best_nn)

# 計算該網路的pre節點輸出的前5筆資料
pre_np = to.get(best_nn_node.pre)
pre_np[:5]

array([[0.87720898, 0.05588952, 0.0669015 ],
       [0.7742843 , 0.10659901, 0.11911669],
       [0.8323601 , 0.08256885, 0.08507105],
       [0.82487579, 0.10501134, 0.07011287],
       [0.89265836, 0.05284318, 0.05449846]])

## 完整代碼

In [15]:
from nyto import net_tool as to
from nyto import layer
from nyto import unit_function as uf
from nyto import train_tool as train

from sklearn import datasets
import numpy as np

# 取得訓練資料
iris=datasets.load_iris()     # 載入資料
feature=iris.data             # 輸入的特徵資料
label=np.eye(3)[iris.target]  # 預測的類別資料

# 導入單元
(nn, node) = to.new_net(                  
    data_x = to.add_data(feature),  # 加入資料節點: data_x
    data_y = to.add_data(label),    # 加入資料節點: data_y
    
    layer1 = layer.new_nn_layer((4,12)),  # 加入模型節點: layer1(單層神經網路)
    layer2 = layer.new_nn_layer((12,3))   # 加入模型節點: layer2(單層神經網路)
)

# 建立節點連結
node.layer1_output = node.data_x >> uf.col_nor() >> node.layer1 >> uf.tanh()
node.layer2_output = node.layer1_output >> node.layer2 >> uf.softmax()
node.pre = node.layer2_output
node.loss = uf.cross_entropy(node.pre, node.data_y)

# 建立池
pool = train.new_pool(node.loss, pool_size=10)

# 訓練池
optimizer=train.epso_opt()
for t in range(10):
    pool = optimizer(pool)
    
# 取得訓練好的網路和該網路的節點
best_nn = pool.net[0]
best_nn_node = to.create_connecter(best_nn)

到這裡基本上已經介紹完如何構建一個類神經網路並訓練了，當在使用nyto面對不同類型的問題時，只需要修改模型或是連接方式的部份就可以處理大部分的狀況了。但是如果想要使網路能更有效率的運作，或是需要更靈活的架構，甚至是了解到更細節的參數調整。

那麼可以查閱後面章節的進階技巧。

***
*END*